In [96]:
import pandas as pd
from boruta import BorutaPy
from sklearn.ensemble      import RandomForestRegressor

# 1.0 Load Data


In [9]:
dscombined = pd.read_csv( 'C:/Users/PICHAU/Desktop/AIbox-Feedback/AIBOX-Mlfeedback/datasets/combined.csv' )
dsLIWC = pd.read_csv( 'C:/Users/PICHAU/Desktop/AIbox-Feedback/AIBOX-Mlfeedback/datasets/combinedLIWC.csv' )
dsCohmetrix = pd.read_csv( 'C:/Users/PICHAU/Desktop/AIbox-Feedback/AIBOX-Mlfeedback/datasets/combined_CohMetrix.csv' )


# 2.0 Merging data

In [20]:
dsCohmetrix = dsCohmetrix.rename( columns={ 'ID':'id' } )

In [24]:
mergedData = pd.merge(dsCohmetrix, dsLIWC, how='left', on='id')
mergedData = pd.merge(mergedData, dscombined, how='left', on='id')
# df1.merge(df2, how='inner', on='a')

In [25]:
mergedData

,id,cm.DESPC,cm.DESSC,cm.DESWC,cm.DESPL,cm.DESPLd,cm.DESSL,cm.DESSLd,cm.DESWLsy,cm.DESWLsyd,...,liwc.leisure,liwc.home,liwc.money,liwc.relig,liwc.death,liwc.assent,liwc.nonfl,liwc.filler,feedback,classe
0,23,1,1,18,1.0,0.0,17.000000,0.000000,1.888889,0.900254,...,1,0,0,0,0,0,0,0,Ler outros textos sobre o assunto para lher da...,0
1,24,1,2,24,2.0,0.0,10.500000,0.500000,1.916667,1.100066,...,1,0,1,0,0,0,0,0,Procure ler mais; para escrever mais acerca do...,0
2,35,1,2,31,2.0,0.0,12.500000,0.500000,1.741935,0.964978,...,2,0,0,0,0,0,0,0,Texto bom; precisa apenas ler mais um pouco; p...,0
3,37,1,1,7,1.0,0.0,6.000000,0.000000,1.857143,1.069045,...,0,0,0,0,0,0,0,0,Ter cuidado com copias da internet.,0
4,68,1,4,32,4.0,0.0,6.500000,3.640055,1.937500,1.268413,...,0,0,1,0,0,2,0,0,Carssimo; o trabalho precisa de uma sistematiz...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,13809,1,2,28,2.0,0.0,12.000000,11.000000,2.250000,1.404358,...,0,0,1,0,0,0,1,0,Rosivalda; timo texto; suas reflexes comparati...,1
996,13810,1,2,66,2.0,0.0,29.500000,27.500000,1.969697,1.149845,...,2,0,1,0,0,1,0,0,Ruth; bom texto sobre o filme e sobre as suas ...,1
997,13811,1,2,37,2.0,0.0,16.500000,15.500000,2.027027,1.213042,...,2,2,0,0,0,1,2,0,Silmara; excelente texto e leitura do filme co...,1
998,13813,1,3,28,3.0,0.0,7.666667,10.143416,1.964286,1.170063,...,2,0,0,1,0,0,1,0,Sueli; boa leitura do filme e da realidade; su...,1


# 3.0 Data Preparation



## 3.1 Normalization

In [37]:
# no analyzed variable looks like a normal distribution

## 3.2 Rescaling

In [32]:
mergedData.dtypes

id               int64
cm.DESPC         int64
cm.DESSC         int64
cm.DESWC         int64
cm.DESPL       float64
                ...   
liwc.assent      int64
liwc.nonfl       int64
liwc.filler      int64
feedback        object
classe           int64
Length: 161, dtype: object

In [36]:
visu = mergedData.select_dtypes( exclude=['int64', 'float64','int32','UInt32'] )
print(visu)

                                              feedback
0    Ler outros textos sobre o assunto para lher da...
1    Procure ler mais; para escrever mais acerca do...
2    Texto bom; precisa apenas ler mais um pouco; p...
3                  Ter cuidado com copias da internet.
4    Carssimo; o trabalho precisa de uma sistematiz...
..                                                 ...
995  Rosivalda; timo texto; suas reflexes comparati...
996  Ruth; bom texto sobre o filme e sobre as suas ...
997  Silmara; excelente texto e leitura do filme co...
998  Sueli; boa leitura do filme e da realidade; su...
999  Taciana; boa leitura do filme e da realidade; ...

[1000 rows x 1 columns]


In [39]:
# thus, due to feedback be a no-analyzed variable, merged dataset doesn't need rescaling
# so, the other variables doesnt need a changes, for now, because the outliers problem will be solved on cohmetrix software  

## 3.3 Transformation


In [51]:
# once time again, transformation is not necessary, due to mergedData only have feedback that is a categorical variable and the response variable is only 0 and 1

# 4.0 Feature Selection


## 4.1 Split dataframe into training and test dataset

In [88]:
# training dataset
X_train = mergedData[mergedData['id']<12800] # this id split dataset in 3/4 proportion
y_train = X_train['classe']

# test dataset 
X_test = mergedData[mergedData['id']>=12800] # this id split dataset in 3/4 proportion
y_test = X_test['classe']


## 4.2 Boruta as Feature Selector

In [110]:
# training and test dataset for Boruta
X_train_n = X_train.drop( ['classe', 'feedback'], axis=1 ).values 
y_train_n = y_train.values.ravel() 

# define RandomForestRegressor
rf = RandomForestRegressor( n_jobs=-1 )

# define Boruta
boruta = BorutaPy( rf, n_estimators='auto', verbose=2, random_state=42 ).fit( X_train_n, y_train_n )


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [114]:
X_train.drop( ['classe', 'feedback', 'id'], axis=1 ).select_dtypes( include=['float64'] )

,cm.DESPL,cm.DESPLd,cm.DESSL,cm.DESSLd,cm.DESWLsy,cm.DESWLsyd,cm.DESWLlt,cm.DESWLltd,cm.CRFCWO1,cm.CRFCWO1d,...,cm.WRDFRQa,cm.WRDFRQmc,cm.WRDAOAc,cm.WRDFAMc,cm.WRDCNCc,cm.WRDIMGc,cm.WRDMEAc,cm.RDFRE,cm.RDFKGL,cm.RDL2
0,1.0,0.0,17.000000,0.000000,1.888889,0.900254,5.117647,2.193100,NaN,NaN,...,1.813555,6.132208,464.666667,559.000000,356.142857,339.285714,336.250000,25.356471,13.945882,NaN
1,2.0,0.0,10.500000,0.500000,1.916667,1.100066,5.095238,2.670915,0.000000,0.000000,...,1.893142,6.308187,375.000000,588.666667,303.000000,323.444444,346.000000,27.485000,11.910000,95.041286
2,2.0,0.0,12.500000,0.500000,1.741935,0.964978,5.040000,2.424541,0.000000,0.000000,...,1.892192,6.357485,403.000000,590.090909,312.333333,339.818182,375.000000,32.505000,11.706667,96.135954
3,1.0,0.0,6.000000,0.000000,1.857143,1.069045,4.833333,2.266912,NaN,NaN,...,1.887103,6.600221,269.000000,561.000000,422.500000,413.500000,349.500000,31.545000,10.350000,NaN
4,4.0,0.0,6.500000,3.640055,1.937500,1.268413,5.500000,3.388328,0.020833,0.029463,...,1.859837,5.138150,486.000000,599.444444,245.375000,264.666667,293.571429,17.755250,12.335500,122.581573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
757,3.0,0.0,6.333333,5.557777,2.120000,1.394035,5.894737,3.193657,0.000000,0.000000,...,1.784089,5.014023,383.000000,605.500000,275.500000,307.500000,372.750000,-8.867018,16.069474,106.787886
758,2.0,0.0,5.500000,4.500000,2.230769,1.832750,6.363636,4.810457,0.000000,0.000000,...,1.802179,5.596473,383.000000,598.500000,222.333333,253.166667,320.500000,11.514423,13.268077,79.237693
759,1.0,0.0,45.000000,0.000000,1.938776,1.375502,4.777778,3.495323,0.030934,0.047533,...,1.864420,6.040351,547.000000,587.222222,270.055556,297.111111,323.882353,-4.670000,25.586809,90.709702
